In [1]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


C:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

In [3]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

In [4]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()

In [5]:
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def main():
    env = gym.make('CartPole-v1')
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s = env.reset()
        done = False

        while not done:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon)      
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask))
            s = s_prime

            score += r
            if done:
                break
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()

n_episode :20, score : 10.2, n_buffer : 205, eps : 7.9%
n_episode :40, score : 9.5, n_buffer : 395, eps : 7.8%
n_episode :60, score : 9.9, n_buffer : 594, eps : 7.7%
n_episode :80, score : 9.7, n_buffer : 787, eps : 7.6%
n_episode :100, score : 9.6, n_buffer : 978, eps : 7.5%
n_episode :120, score : 10.1, n_buffer : 1179, eps : 7.4%
n_episode :140, score : 9.6, n_buffer : 1371, eps : 7.3%
n_episode :160, score : 9.9, n_buffer : 1569, eps : 7.2%
n_episode :180, score : 10.1, n_buffer : 1770, eps : 7.1%
n_episode :200, score : 10.1, n_buffer : 1971, eps : 7.0%


C:\Users\user\AppData\Local\Temp\ipykernel_9712\848517094.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \


n_episode :220, score : 16.0, n_buffer : 2291, eps : 6.9%
n_episode :240, score : 11.8, n_buffer : 2528, eps : 6.8%
n_episode :260, score : 10.3, n_buffer : 2735, eps : 6.7%
n_episode :280, score : 11.7, n_buffer : 2968, eps : 6.6%
n_episode :300, score : 15.2, n_buffer : 3272, eps : 6.5%
n_episode :320, score : 22.9, n_buffer : 3729, eps : 6.4%
n_episode :340, score : 63.8, n_buffer : 5004, eps : 6.3%
n_episode :360, score : 76.0, n_buffer : 6525, eps : 6.2%
n_episode :380, score : 87.3, n_buffer : 8271, eps : 6.1%
n_episode :400, score : 93.5, n_buffer : 10142, eps : 6.0%
n_episode :420, score : 124.7, n_buffer : 12635, eps : 5.9%
n_episode :440, score : 189.9, n_buffer : 16433, eps : 5.8%
n_episode :460, score : 226.4, n_buffer : 20961, eps : 5.7%
n_episode :480, score : 251.0, n_buffer : 25981, eps : 5.6%
n_episode :500, score : 210.2, n_buffer : 30185, eps : 5.5%
n_episode :520, score : 171.7, n_buffer : 33618, eps : 5.4%
n_episode :540, score : 156.7, n_buffer : 36752, eps : 5.3%

: 

: 